# Quick Start

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/osllmai/inDox/blob/master/Demo/quick_start.ipynb)

In [ ]:
!pip install indox
!pip install openai
!pip install chromadb

In [4]:
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.environ['OPENAI_API_KEY']

## Initial Setup

The following imports are essential for setting up the Indox application. These imports include the main Indox retrieval augmentation module, question-answering models, embeddings, and data loader splitter.

In [5]:
from indox import IndoxRetrievalAugmentation
indox = IndoxRetrievalAugmentation()

2024-07-02 10:02:41,625 INFO:IndoxRetrievalAugmentation initialized


### Generating response using OpenAI's language models 
OpenAIQA class is used to handle question-answering task using OpenAI's language models. This instance creates OpenAiEmbedding class to specifying embedding model. Here ChromaVectorStore handles the storage and retrieval of vector embeddings by specifying a collection name and sets up a vector store where text embeddings can be stored and queried.

In [6]:
from indox.llms import OpenAi
from indox.embeddings import OpenAiEmbedding
openai_qa = OpenAi(api_key=OPENAI_API_KEY, model="gpt-3.5-turbo-0125")
embed_openai = OpenAiEmbedding(api_key=OPENAI_API_KEY,model="text-embedding-3-small")

from indox.vector_stores import ChromaVectorStore
db = ChromaVectorStore(collection_name="sample",embedding=embed_openai)
indox.connect_to_vectorstore(vectorstore_database=db)

2024-07-02 10:02:43,114 INFO:Initializing OpenAi with model: gpt-3.5-turbo-0125
2024-07-02 10:02:43,711 INFO:OpenAi initialized successfully
2024-07-02 10:02:45,567 INFO:Initialized OpenAI embeddings with model: text-embedding-3-small
2024-07-02 10:02:45,990 INFO:Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
2024-07-02 10:02:46,126 INFO:Attempting to connect to the vector store database
2024-07-02 10:02:46,128 INFO:Connection to the vector store database established successfully


### load and preprocess data
This part of code demonstrates how to load and preprocess text data from a file, split it into chunks, and store these chunks in the vector store that was set up previously.

In [ ]:
!wget https://raw.githubusercontent.com/osllmai/inDox/master/Demo/sample.txt

In [7]:
file_path = "sample.txt"

In [8]:
from indox.data_loader_splitter import UnstructuredLoadAndSplit
loader_splitter = UnstructuredLoadAndSplit(file_path=file_path,max_chunk_size=400)
docs = loader_splitter.load_and_chunk()

2024-07-02 10:02:47,993 INFO:Backing off send_request(...) for 0.5s (requests.exceptions.SSLError: HTTPSConnectionPool(host='us-api.i.posthog.com', port=443): Max retries exceeded with url: /batch/ (Caused by SSLError(SSLError(1, '[SSL: WRONG_VERSION_NUMBER] wrong version number (_ssl.c:1000)'))))
2024-07-02 10:02:49,836 INFO:Backing off send_request(...) for 0.2s (requests.exceptions.SSLError: HTTPSConnectionPool(host='us-api.i.posthog.com', port=443): Max retries exceeded with url: /batch/ (Caused by SSLError(SSLError(1, '[SSL: WRONG_VERSION_NUMBER] wrong version number (_ssl.c:1000)'))))
2024-07-02 10:02:50,790 INFO:Backing off send_request(...) for 2.8s (requests.exceptions.SSLError: HTTPSConnectionPool(host='us-api.i.posthog.com', port=443): Max retries exceeded with url: /batch/ (Caused by SSLError(SSLError(1, '[SSL: WRONG_VERSION_NUMBER] wrong version number (_ssl.c:1000)'))))
2024-07-02 10:02:54,022 INFO:Initializing UnstructuredLoadAndSplit
2024-07-02 10:02:54,022 INFO:Unstruc

In [9]:
indox.store_in_vectorstore(docs=docs)

2024-07-02 10:02:58,565 INFO:Storing documents in the vector store
2024-07-02 10:03:01,823 INFO:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-07-02 10:03:04,140 INFO:Document added successfully to the vector store.
2024-07-02 10:03:04,141 INFO:Documents stored successfully


### Retrieve relevant information and generate an answer
The main purpose of these lines is to perform a query on the vector store to retrieve the most relevant information (top_k=5) and generate an answer using the language model.

In [10]:
query = "How Cinderella reach her happy ending?"
retriever = indox.QuestionAnswer(vector_database=db, llm=openai_qa, top_k=5)

invoke(query) method sends the query to the retriever, which searches the vector store for relevant text chunks and uses the language model to generate a response based on the retrieved information.
Context property retrieves the context or the detailed information that the retriever used to generate the answer to the query. It provides insight into how the query was answered by showing the relevant text chunks and any additional information used.

In [21]:
answer = retriever.invoke(query)
context = retriever.context

2024-07-02 10:07:35,768 INFO:Retrieving context and scores from the vector database
2024-07-02 10:07:37,064 INFO:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-07-02 10:07:37,069 INFO:Generating answer without document relevancy filter
2024-07-02 10:07:37,070 INFO:Answering question
2024-07-02 10:07:37,071 INFO:Generating response
2024-07-02 10:07:40,970 INFO:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-07-02 10:07:40,972 INFO:Response generated successfully
2024-07-02 10:07:40,972 INFO:Query answered successfully


### With AgenticRag

AgenticRag stands for Agentic Retrieval-Augmented Generation. This concept combines retrieval-based methods and generation-based methods in natural language processing (NLP). The key idea is to enhance the generative capabilities of a language model by incorporating relevant information retrieved from a database or a vector store. 
 AgenticRag is designed to provide more contextually rich and accurate responses by utilizing external knowledge sources. It retrieves relevant pieces of information (chunks) from a vector store based on a query and then uses a language model to generate a comprehensive response that incorporates this retrieved information.

In [10]:
agent = indox.AgenticRag(llm=openai_qa,vector_database=db,top_k=5)
agent.run(query)

Not Relevant doc
Relevant doc
Not Relevant doc
Not Relevant doc
Not Relevant doc


"Cinderella reached her happy ending by receiving a branch from the hazel-bush from the prince, planting it on her mother's grave, and weeping and praying beneath it. The branch grew into a handsome tree, and a little white bird always came to the tree, bringing her comfort and assistance."

## Evaluation

In [27]:
from indox.evaluation import Evaluation
evaluator = Evaluation(["BertScore", "Toxicity",  "Reliability", "Fairness" , "Readibility"])

2024-07-02 10:27:22,324 INFO:Use pytorch device: cpu


In [28]:
inputs = {
    "question" : query,
    "answer" : answer,
    "context" : context
}
result = evaluator(inputs)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [29]:
result

,0
Precision,0.493488
Recall,0.517235
F1-score,0.505083
Toxicity,0.081307
hallucination_score,0.620000
Fairness,0.496180
Perplexity,38.828209
ARI,12.800000
Flesch-Kincaid Grade Level,10.100000
